#### Data 612 - Project 2 : Content-Based and Collaborative Filtering<br>Date: June 18, 2019<br>Team Info: 
+ Christina Valore
+ Juliann McEachern 
+ Rajwant Mishra

<h1 align="center">Good Books Recommender System</h1>

## Dataset Selection

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load data from local csv  into pandas dataframe
book_tags = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/book_tags.csv')
tags = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/tags.csv')
books = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/data/books.csv')


BadZipFile: File is not a zip file

In [19]:
book_tags.head()
ratings=pd.read_csv('https://github.com/jemceach/612-group/blob/master/project-2/data/ratings.csv', compression='gzip')

HTTPError: HTTP Error 404: Not Found

In [3]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [10]:
np.shape(books)

(10000, 23)

In [16]:
np.shape(ratings)

ratings.to_csv('data/ratings.csv')

## Content-Based Filtering 

## User-User Collaborative Filtering 

## Item-Item Collaborative Filtering 

## Summary
Please provide at least one graph, and a textual summary of your findings and recommendations. 